In [1]:
!pip3 install quantlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 20.8 MB/s eta 0:00:0000:0100:01


![image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTt1c0Ihlmsjkb4JNC4J32EkcLZjd-itwIFLg&usqp=CAU)

In [3]:
import QuantLib as ql

In [177]:
# 평가일 지정 날짜 할당
valuation_date = ql.Date(1, 6, 2021)

# 평가일 기준일을 위서 설정한 날짜로 할당
ql.Settings.instance().evaluationDate = valuation_date

# 날짜 계산을 위한 영업일 수 계산으로 한국 달력 사용
calendar = ql.SouthKorea()
dayCount = ql.ActualActual()

In [179]:
# 기초자산 가격, 배당률, 무위험이자율(ex. 기준금리), 변동성 지수 업데이트
underlying_qt = ql.SimpleQuote(270)
dividend_qt = ql.SimpleQuote(0.0)
riskfreerate_qt = ql.SimpleQuote(0.0125)
volatility_qt = ql.SimpleQuote(0.13)

In [180]:
# 업데이트한 SimpleQuote를 다시 QuoteHandle로 설정하는 부분
# 이 작업을 토대로 simple 값이 변화하더라도 handle을 통해 바로 값 변형이 반영됨
# Handle은 여기서 메모리 address 값을 저장하는 pointer 역할을 수행한다.
q_qhandle = ql.QuoteHandle(underlying_qt)
d_qhandle = ql.QuoteHandle(dividend_qt)
r_qhandle = ql.QuoteHandle(riskfreerate_qt)
v_qhandle = ql.QuoteHandle(volatility_qt)

In [181]:
# 기초자산 가격을 제외한 나머지 변수는 기간 구조로 변경해야 함.

r_timestructure = ql.FlatForward(valuation_date, r_qhandle, dayCount)
d_timestructure = ql.FlatForward(valuation_date, d_qhandle, dayCount)
v_timestructure = ql.BlackConstantVol(valuation_date, calendar, v_qhandle, dayCount)

In [182]:
# handle을 통해 address를 넘겨주 듯, 마찬가지로 기간구조도 handle로 넘겨주는 작업
r_thandle = ql.YieldTermStructureHandle(r_timestructure)
d_thandle = ql.YieldTermStructureHandle(d_timestructure)
v_thandle = ql.BlackVolTermStructureHandle(v_timestructure)

In [183]:
# 가격 산정 프로세스 설정 (블렉숄츠 모형에서 가정하는 확률과정에 의한 feature 입력)
process = ql.BlackScholesMertonProcess(q_qhandle, d_thandle, r_thandle, v_thandle)
engine = ql.AnalyticEuropeanEngine(process)

In [184]:
# 2020년 12월 12일 만기고 call 행사가격은 272인 유럽형 콜옵션 상정
option_type = ql.Option.Call
strike_price = 272
expiry_date = ql.Date(1,12,2021)
exercise = ql.EuropeanExercise(expiry_date)
payoff = ql.PlainVanillaPayoff(type= option_type, strike = strike_price)
option = ql.VanillaOption(payoff=payoff, exercise=exercise)

In [185]:
# 옵션 가격 설정을 하기 위한 마지막 작업
option.setPricingEngine(engine)

In [186]:
# 데이터 출력
result_value = [option.NPV(), option.delta(), option.gamma(), option.thetaPerDay(), option.vega()/100, option.rho()/100]
result_type = ['옵션프리미엄', '옵션 델타', '옵션 감마', '옵션 세타', '옵션 베가', '옵션 로']
for i,j in zip(result_type, result_value):
    print(f'{i} = {round(j, 8)}')

옵션프리미엄 = 9.76748896
옵션 델타 = 0.51353501
옵션 감마 = 0.01604256
옵션 세타 = -0.03148873
옵션 베가 = 0.76225959
옵션 로 = 0.64620039


In [187]:
# 기초자산의 가격이 변동하는 경우
underlying_qt.setValue(275)
# 데이터 출력
result_value = [option.NPV(), option.delta(), option.gamma(), option.thetaPerDay(), option.vega()/100, option.rho()/100]
result_type = ['옵션프리미엄', '옵션 델타', '옵션 감마', '옵션 세타', '옵션 베가', '옵션 로']
for i,j in zip(result_type, result_value):
    print(f'{i} = {round(j, 4)}')

옵션프리미엄 = 12.5334
옵션 델타 = 0.5922
옵션 감마 = 0.0153
옵션 세타 = -0.032
옵션 베가 = 0.756
옵션 로 = 0.7537


In [188]:
market_price = 8.21
implied_volatility = option.impliedVolatility(market_price, process)
volatility_qt.setValue(implied_volatility)

result_value = [option.NPV(), option.delta(), option.gamma(), option.thetaPerDay(), option.vega()/100, option.rho()/100]
result_type = ['옵션프리미엄', '옵션 델타', '옵션 감마', '옵션 세타', '옵션 베가', '옵션 로']

for i,j in zip(result_type, result_value):
    print(f'{i} = {round(j, 4)}')

옵션프리미엄 = 8.2097
옵션 델타 = 0.6419
옵션 감마 = 0.0266
옵션 세타 = -0.0201
옵션 베가 = 0.7272
옵션 로 = 0.8438
